In [1]:
import pickle
from collections import namedtuple
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import pandas as pd
import sys
sys.path.append("/home/angrypark/korean-text-matching-tf/")

from utils.utils import JamoProcessor
from text.tokenizers import SentencePieceTokenizer

Config = namedtuple("config", ["sent_piece_model"])
config = Config("/media/scatter/scatterdisk/tokenizer/sent_piece.100K.model")
processor = JamoProcessor()
tokenizer = SentencePieceTokenizer(config)

def my_word_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [word for word in tokenizer.tokenize(raw)]

def my_char_tokenizer(raw, pos=["Noun", "Alpha", "Verb", "Number"], stopword=[]):
    return [processor.word_to_jamo(word) for word in tokenizer.tokenize(raw)]

In [2]:
from feature_extractor import FeatureExtractor
feature_extractor = FeatureExtractor()

Pre-trained embedding loaded. Number of OOV : 5272 / 90000


/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /media/scatter/scatterdisk/reply_matching_model/runs/delstm_1024_nsrandom4_lr1e-3/best_loss/best_loss.ckpt


In [39]:
def model_to_submission(model, name):
    def load_data():
        base_dir = "/home/angrypark/paraphrase_detection/data/origin_data/"
        train_set = pd.read_csv(base_dir + "train_set.csv")
        val_set = pd.read_csv(base_dir + "val_set.csv")
        test_set = pd.read_csv(base_dir + "test_set.csv")
        with open(base_dir + "train.txt", "r") as f:
            _, _, train_labels = zip(*[line.strip().split("\t") for line in f if line])
            train_labels = [1 if l=="1" else 0 for l in train_labels]
        with open(base_dir + "val.txt", "r") as f:
            _, _, val_labels = zip(*[line.strip().split("\t") for line in f if line]) 
            val_labels = [1 if l=="1" else 0 for l in val_labels]
        with open(base_dir + "test.txt", "r") as f:
            _, _, test_labels = zip(*[line.strip().split("\t") for line in f if line])
            test_labels = [1 if l=="1" else 0 for l in test_labels]
            
        return train_set, val_set, test_set, train_labels, val_labels, test_labels
    
    with open("/home/angrypark/paraphrase_detection/data/test_queries.txt", "r") as f:
        _, queries = zip(*[line.strip().split("\t") for line in f])
        
    with open("/home/angrypark/paraphrase_detection/data/test_replies.txt", "r") as f:
        reply_to_idx_dict = dict()
        replies = list()
        for line in f:
            splits = line.strip().split("\t")
            reply_to_idx_dict[splits[1]] = splits[0]
            replies.append(splits[1])
    
    train_set, val_set, test_set, train_labels, val_labels, test_labels = load_data()
    print("Train accuracy : {:.8f}".format(model.score(train_set, train_labels)))
    print("Val   accuracy : {:.8f}".format(model.score(val_set, val_labels)))
    print("Test  accuracy : {:.8f}".format(model.score(test_set, test_labels)))
    
    length = len(replies)
    submit_set = pd.read_csv("../data/submit_set_update.csv")
    features = list(submit_set.columns).copy()
    features.remove("sentence_A")
    features.remove("sentence_B")
    
    predictions = list()
    for i, query in enumerate(queries):
        data = submit_set.iloc[i*length:(i+1)*length]
        probs = [p[1] for p in model.predict_proba(data[features]).tolist()]
        scores = [(reply, score) for reply, score in zip(replies, probs)]
        predict = sorted(scores, key=lambda x: x[1], reverse=True)[0][0]
        predictions.append(reply_to_idx_dict[predict])
    submission = pd.read_csv("../submission/sample_submission.csv")
    submission["id_script"] = pd.Series(predictions)
    submission.to_csv("../submission/{}.csv".format(name), index=False)
    return submission

In [27]:
with open("../data/pure/train.txt", "r") as f:
    train_A, train_B, labels = zip(*[line.strip().split("\t") for line in f])
    train_labels = [1 if l=="1" else 0 for l in labels]

In [28]:
with open("../data/pure/val.txt", "r") as f:
    val_A, val_B, labels = zip(*[line.strip().split("\t") for line in f])
    val_labels = [1 if l=="1" else 0 for l in labels]
    
with open("../data/pure/test.txt", "r") as f:
    test_A, test_B, labels = zip(*[line.strip().split("\t") for line in f])
    test_labels = [1 if l=="1" else 0 for l in labels]

In [29]:
%%time
train_set = pd.DataFrame(feature_extractor.extract_features(train_A, train_B))
train_set.to_csv("../data/pure/train_set.csv", index=False)

CPU times: user 7min 48s, sys: 16.8 s, total: 8min 5s
Wall time: 7min 20s


In [30]:
%%time
val_set = pd.DataFrame(feature_extractor.extract_features(val_A, val_B))
val_set.to_csv("../data/pure/val_set.csv", index=False)

CPU times: user 1min 7s, sys: 2.8 s, total: 1min 10s
Wall time: 1min 3s


In [31]:
%%time
test_set = pd.DataFrame(feature_extractor.extract_features(test_A, test_B))
test_set.to_csv("../data/pure/test_set.csv", index=False)

CPU times: user 42.3 s, sys: 1.4 s, total: 43.7 s
Wall time: 39.6 s


In [32]:
total = pd.concat([pd.read_csv("../data/pure/train_set.csv"), pd.read_csv("../data/pure/val_set.csv")])

In [33]:
total_labels = train_labels + val_labels

pure + train&val + tfidf update

In [34]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(total, total_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [38]:
_ = model_to_submission(xgb, name="xgb_pure")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85066667
Val   accuracy : 0.73891881
Test  accuracy : 0.73543016


120000it [02:17, 874.83it/s]


pure + train + tfidf word update

In [40]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(train_set, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [41]:
_ = model_to_submission(xgb, name="xgb_pure")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.84969000
Val   accuracy : 0.73632974
Test  accuracy : 0.73846967


---
어제 1등한 모델

In [23]:
_ = model_to_submission(xgb_old, name="xgb_old")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85188855
Val   accuracy : 0.74361713
Test  accuracy : 0.72810643


---
내가 개난리 치면서 라벨링 해서 추가했지만 성능은 더 나빠진 모델

In [38]:
_ = model_to_submission(xgb, name="xgb_new")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85182910
Val   accuracy : 0.74295596
Test  accuracy : 0.72746060


---
방금 그대로 다시 학습

In [48]:
_ = model_to_submission(xgb, name="xgb_old_2")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85096333
Val   accuracy : 0.73990265
Test  accuracy : 0.72327210


In [4]:
train_set = pd.read_csv("../data/small/train_set.csv")
val_set = pd.read_csv("../data/small/val_set.csv")
test_set = pd.read_csv("../data/small/test_set.csv")

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
word_sim = list()
for a, b in zip(train_A, train_B):
    word_sim.append(cosine_similarity(feature_extractor.tfidf_word_vectorizer.transform([a]), 
                                      feature_extractor.tfidf_word_vectorizer.transform([b]))[0][0])
train_set["tfidf_word_sim"] = pd.Series(word_sim)

In [12]:
word_sim = list()
for a, b in zip(val_A, val_B):
    word_sim.append(cosine_similarity(feature_extractor.tfidf_word_vectorizer.transform([a]), 
                                      feature_extractor.tfidf_word_vectorizer.transform([b]))[0][0])
val_set["tfidf_word_sim"] = pd.Series(word_sim)

In [13]:
word_sim = list()
for a, b in zip(test_A, test_B):
    word_sim.append(cosine_similarity(feature_extractor.tfidf_word_vectorizer.transform([a]), 
                                      feature_extractor.tfidf_word_vectorizer.transform([b]))[0][0])
test_set["tfidf_word_sim"] = pd.Series(word_sim)

In [14]:
total = pd.concat([train_set, val_set])
total_labels = pd.Series(train_labels + val_labels)

tfidf word vectorizer 업데이트

In [19]:
xgb = XGBClassifier()
xgb.fit(total, total_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [20]:
print("Train : {:.8f}".format(xgb.score(train_set, train_labels)))
print("Val   : {:.8f}".format(xgb.score(val_set, val_labels)))
print("Test  : {:.8f}".format(xgb.score(test_set, test_labels)))

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train : 0.85105631
Val   : 0.74173533
Test  : 0.72487729


---
train set에만 학습한다면?

In [17]:
xgb = XGBClassifier()
xgb.fit(train_set, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [18]:
print("Train : {:.8f}".format(xgb.score(train_set, train_labels)))
print("Val   : {:.8f}".format(xgb.score(val_set, val_labels)))
print("Test  : {:.8f}".format(xgb.score(test_set, test_labels)))

Train : 0.85142950
Val   : 0.73354694
Test  : 0.72287523


/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
from tqdm import tqdm

In [24]:
_ = model_to_submission(xgb, name="xgb_word_update")

/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/angrypark/angryenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Train accuracy : 0.85019667
Val   accuracy : 0.73845278
Test  accuracy : 0.72089335


120000it [02:21, 847.83it/s]


In [26]:
train_set.to_csv("../data/small/train_set.csv", index=False)
val_set.to_csv("../data/small/val_set.csv", index=False)
test_set.to_csv("../data/small/test_set.csv", index=False)

In [ ]:
train_set2 = pd.read_csv("../data/small/")

In [ ]:
from catboost import CatBoostClassifier
cb = CatBoostClassifier()
cb.fit(train_set, train_labels)